In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

In [3]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [4]:
df_get_back=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\get back.xlsx')
print(df_get_back.shape)
df_get_back.head()

df_get_back=df_get_back.loc[:,['id','lon','lat','check']]

(133014, 6)


In [5]:
pop=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\台灣202008年村里人口.xlsx')
print(pop.shape)

(7760, 5)


In [6]:
site=['1G30林口中山店']
date=['202009']
start=['190726']

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir('C:/Users/RT009223/TestriteGroup/滲透率專案/01_raw data/TLW社區店/'+site[0]+'/'+date[0])

In [7]:
df01=pd.read_csv(r'ff交易紀錄.csv',header=None,names=['id','site','date','AMT','trans_cnt','tick_cnt','card_type'])
print(df01.shape)
print(df01.head())

df01['temp']=0
df01.loc[df01.card_type=='05',['temp']]=1

df01_2=df01.groupby(['site','temp'])['AMT'].sum().reset_index(name='AMT')
df01_2['share']=df01_2.AMT/df01_2.AMT.sum()

df_final01=df01_2.copy()
print(df_final01)

(2090, 7)
              id  site                     date     AMT  trans_cnt  tick_cnt  \
0  2752450434017  1G30  2020-09-13 00:00:00.000   190.0          1         1   
1  2014842400018  1G30  2020-09-04 00:00:00.000  1951.0          1         1   
2  2161178020011  1G30  2020-09-25 00:00:00.000     0.0          0         0   
3  2811916070128  1G30  2020-09-01 00:00:00.000   247.0          1         2   
4  2655000904339  1G30  2020-09-20 00:00:00.000   132.0          1         1   

  card_type  
0        00  
1        00  
2        00  
3        00  
4        00  
   site  temp         AMT     share
0  1G30     0  2132839.97  0.876688
1  1G30     1   299996.83  0.123312


In [8]:
df02=pd.read_csv(r'一般卡會員活躍程度.csv',header=None,names=['id','type'])
print(df02.shape)
print(df02.head())

df02.type=df02.type.apply(lambda x:str(x))
df02.type=df02.type.replace('1','type1:new')
df02.type=df02.type.replace('2','type2:active')
df02.type=df02.type.replace('3','type3:1yr~3yr')
df02.type=df02.type.replace('4','type4:3yr~')

df02_2=df02.groupby(['type'])['id'].count().reset_index(name='n')
df02_2['share']=df02_2.n/df02_2.n.sum()

df_final02=df02_2.copy()
print(df_final02)

(1596, 2)
               id  type
0  19960129000912     2
1  19960202000788     2
2  19960203000289     2
3  19960205002424     2
4  19960206000452     2
            type     n     share
0      type1:new    67  0.041980
1   type2:active  1152  0.721805
2  type3:1yr~3yr   115  0.072055
3     type4:3yr~   262  0.164160


In [9]:
df03=pd.read_csv(r'一般卡會員會員樣貌.csv',header=None,names=['id','name','gender','birth','status','age','age_type'])
print(df03.shape)
print(df03.head())


df03=pd.merge(df03,df02,on='id')

df03_2=df03.groupby(['age_type'])['id'].count().reset_index(name='n_all')
df03_2['share_all']=df03_2.n_all/df03_2.n_all.sum()
df03_3=df03[df03.type=='type1:new'].groupby(['age_type'])['id'].count().reset_index(name='n_new')
df03_3['share_new']=df03_3.n_new/df03_3.n_new.sum()

df03_4=pd.merge(df03_2,df03_3,on='age_type',how='outer')
df03_4=df03_4.fillna(0)
print(df03_4.share_all.sum())
print(df03_4.share_new.sum())

df_final03=df03_4.copy()
print(df_final03)

(1596, 7)
               id   name gender                    birth  status   age  \
0  20000109003490    李明忠      M  1971-05-02 00:00:00.000       1  49.0   
1  20060811000097    宋海舟      M  1970-10-01 00:00:00.000       1  49.0   
2  19991221000046  陳邱珍雅.      F  1961-01-01 00:00:00.000       1  59.0   
3  19990919000735    馬增松      M  1961-08-11 00:00:00.000       2  59.0   
4  20060603000445    吳欣舜      M  1952-07-01 00:00:00.000       1  68.0   

   age_type  
0  08:45-49  
1  08:45-49  
2  10:55-59  
3  10:55-59  
4  12:65-69  
1.0
1.0
    age_type  n_all  share_all  n_new  share_new
0   02:15-19     10   0.006266    3.0   0.044776
1   03:20-24     25   0.015664   11.0   0.164179
2   04:25-29     57   0.035714    8.0   0.119403
3   05:30-34     97   0.060777    4.0   0.059701
4   06:35-39    199   0.124687    5.0   0.074627
5   07:40-44    266   0.166667    5.0   0.074627
6   08:45-49    226   0.141604    5.0   0.074627
7   09:50-54    209   0.130952    1.0   0.014925
8   10:55-59

In [10]:
df03_5=df03.groupby(['gender'])['id'].count().reset_index(name='n_all')
df03_5['share_all']=df03_5.n_all/df03_5.n_all.sum()
df03_6=df03[df03.type=='type1:new'].groupby(['gender'])['id'].count().reset_index(name='n_new')
df03_6['share_new']=df03_6.n_new/df03_6.n_new.sum()

df03_7=pd.merge(df03_5,df03_6,on='gender',how='outer')
df03_7=df03_7.fillna(0)
print(df03_7.share_all.sum())
print(df03_7.share_new.sum())

df_final04=df03_7.copy()
df_final04

1.0
1.0


,gender,n_all,share_all,n_new,share_new
0,F,612,0.383459,31,0.462687
1,M,981,0.614662,35,0.522388
2,Z,3,0.001880,1,0.014925


In [11]:
df04=pd.read_csv(r'sales一般卡交易紀錄.csv',header=None,names=['id','card','site','date','sku','class1','class1_name','AMT'])
print(df04.shape)
df04.head()

(6151, 8)


,id,card,site,date,sku,class1,class1_name,AMT
0,19960707000067,2011056120176,1G30,2020-09-21 00:00:00.000,16172911,1,油漆/防水,562.0
1,19970218000146,2011706800168,1G30,2020-09-06 00:00:00.000,16140185,18,家用品,94.0
2,19970802000112,2012055960176,1G30,2020-09-21 00:00:00.000,16284254,9,衛浴,1048.0
3,19971108000026,2012247140126,1G30,2020-09-18 00:00:00.000,14248707,37,文具,35.0
4,19980125000051,2020265640108,1G30,2020-09-20 00:00:00.000,16033468,2,膠類,52.0


In [12]:
df04_2=pd.merge(df04,df03,on='id',how='left')
print(df04_2.shape)
df04_2.head()

(6151, 15)


,id,card,site,date,sku,class1,class1_name,AMT,name,gender,birth,status,age,age_type,type
0,19960707000067,2011056120176,1G30,2020-09-21 00:00:00.000,16172911,1,油漆/防水,562.0,林志文,M,1900-01-01 00:00:00.000,1,120.0,99:其他,type2:active
1,19970218000146,2011706800168,1G30,2020-09-06 00:00:00.000,16140185,18,家用品,94.0,黃武男,M,1900-01-01 00:00:00.000,1,120.0,99:其他,type4:3yr~
2,19970802000112,2012055960176,1G30,2020-09-21 00:00:00.000,16284254,9,衛浴,1048.0,顏嘉良,M,1962-09-25 00:00:00.000,1,57.0,10:55-59,type2:active
3,19971108000026,2012247140126,1G30,2020-09-18 00:00:00.000,14248707,37,文具,35.0,司龍成,M,1960-03-18 00:00:00.000,1,60.0,11:60-64,type2:active
4,19980125000051,2020265640108,1G30,2020-09-20 00:00:00.000,16033468,2,膠類,52.0,劉育廷,M,1967-11-15 00:00:00.000,1,52.0,09:50-54,type2:active


In [13]:
df04_3=df04_2.groupby(['class1_name'])['id'].nunique().reset_index(name='id_all')
df04_3['id_all_rate']=df04_3.id_all/df03.shape[0]
print(df04_3.shape)

df04_4=df04_2[df04_2.type=='type1:new'].groupby(['class1_name'])['id'].nunique().reset_index(name='id_new')
df04_4['id_new_rate']=df04_4.id_new/df03[df03.type=='type1:new'].shape[0]
print(df04_4.shape)
df04_4.head()

(26, 3)
(23, 3)


,class1_name,id_new,id_new_rate
0,五金,9,0.134328
1,休閒樂活&園藝,6,0.089552
2,傢俱,2,0.029851
3,地材,5,0.074627
4,家用品,21,0.313433


In [14]:
dept_name=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\TLW社區店\大分類中文名稱.xlsx')
df04_5=pd.merge(df04_3,df04_4,on='class1_name',how='outer')

df_final05=pd.merge(dept_name,df04_5,how='left')
df_final05=df_final05.fillna(0)
print(df_final05.shape)
df_final05.head()

(92, 5)


,class1_name,id_all,id_all_rate,id_new,id_new_rate
0,油漆/防水,166.0,0.104010,5.0,0.074627
1,膠類,155.0,0.097118,6.0,0.089552
2,窗飾/壁飾,35.0,0.021930,1.0,0.014925
3,地材,57.0,0.035714,5.0,0.074627
4,燈源/燈具,265.0,0.166040,14.0,0.208955


In [15]:
df_01=pd.read_csv('一般卡會員(有完整地址)_'+start[0]+'.csv',error_bad_lines=False,header=None)
df_01.columns=['id','addr','lon','lat']
df_01.shape

(12039, 4)

In [16]:
pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
location=df_01.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [17]:
df_01['縣市']=addr01
df_01['行政區']=addr02
df_01['村里']=addr03

In [18]:
df_02=df_01.dropna()
df_02.shape

(12036, 7)

In [19]:
gdf=gpd.read_file(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\geo data\VILLAGE_MOI_1090423.shp',encoding='utf-8')
gdf=gdf.loc[:,['COUNTYNAME','TOWNNAME','VILLNAME','geometry']]
gdf=trans(gdf)
gdf=gdf.dropna().reset_index(drop=True)
print(gdf.shape)

gdf.columns=['縣市','行政區','村里','geometry']
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
gdf.head()

(7760, 4)


,縣市,行政區,村里,geometry
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498..."
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492..."
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577..."
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557..."
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141..."


In [23]:
site1=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].groupby(['村里'])['行政區'].nunique().reset_index(name='n').sort_values(by='n',ascending=False).reset_index(drop=True)
site1.head(5)

,村里,n
0,公西里,1


In [24]:
for i in site1[site1.n>0]['村里']:
    print(i, df_02[df_02['村里']==i]['行政區'].unique())

公西里 ['龜山區' '林口區']


In [25]:
site2=site1[site1.n>0]['村里'][0]
site3=df_02[df_02['村里']==site2]['行政區'].unique()

for i in site3:
    print(i,list(gdf[gdf['行政區']==i]['村里'][gdf[gdf['行政區']==i]['村里'].str.contains('公西里')]))

龜山區 ['公西里']
林口區 []


In [22]:
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))
#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?里','南瑶里'))


df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?田里','塩田里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('山?里','山脚里'))

df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水?村','水祭村'))
df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水？村','水祭村'))
df_02.loc[df_02['行政區']=='瑞芳區',['村里']]=df_02.loc[df_02['行政區']=='瑞芳區',['村里']].apply(lambda x:x.replace('?洞里','濂洞里'))

df_02.loc[df_02['行政區']=='埔心鄉',['村里']]=df_02.loc[df_02['行政區']=='埔心鄉',['村里']].apply(lambda x:x.replace('埤?村','埤脚村'))

In [26]:
#合
df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

#不合
df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_02_1.shape)
print(df_02_2.shape)

(12035, 7)
(1, 7)


In [27]:
df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))

In [28]:
if df_02_2.shape[0]>0:
    geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
    print(df_02_2.shape)
    df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
    print(df_02_3.shape)
    df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    df_02_3.head()
else:
    df_02_3=df_02_2.copy()

(1, 8)
(1, 12)


In [29]:
df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
df_02.shape

(12036, 7)

In [30]:
df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
df_03.shape

(3, 7)

In [31]:
pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
location=df_03.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [32]:
df_03['縣市']=addr01
df_03['行政區']=addr02
df_03['村里']=addr03

In [33]:
#合
df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_1.shape)

#不合
df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_2.shape)


(1, 7)
(2, 7)


In [34]:
def to_float(x):
    try:
        return float(x)
    except:
        return 0

In [35]:
df_03_2.lon=df_03_2.lon.apply(to_float)
df_03_2.lat=df_03_2.lat.apply(to_float)

if df_03_2.shape[0]>0:
    geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
    print(df_03_2.shape)
    df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
    print(df_03_3.shape)
    df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    print(df_03_3.shape)
else:
    df_03_3=df_03_2.copy()

(2, 8)
(2, 12)
(2, 7)


In [36]:
df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)

In [37]:
df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
df_01.shape

(12039, 7)

In [38]:
df_04=pd.read_excel(r'一般卡會員(無完整地址)_'+start[0]+'.xlsx')
df_04.shape

(2345, 3)

In [39]:
df_04['addr']=df_04.addr.apply(lambda x:str(x))

In [40]:
df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
df_04_1.shape

(1093, 3)

In [41]:
pattern = re.compile(r'..[市,縣].+[路,街].+')
location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

addr01=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0])      
    else:
        addr01.append(np.nan)

In [42]:
df_04_1.addr=addr01
df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
df_04_1.shape

(881, 3)

In [43]:
df_04_1['lon']=float(0)
df_04_1['lat']=float(0)
df_04_1['check']=0
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20051121000155,台北市中山區南京都路３段１２９號９樓,104.0,0.0,0.0,0
1,20000430000961,新北市八里區商港路96號14樓,249.0,0.0,0.0,0
2,20020306000771,彰化縣北斗鎮大新里路１６７巷１９號,521.0,0.0,0.0,0
3,20000401009236,新北市林口區中湖路39號之,244.0,0.0,0.0,0
4,20031217000642,新北市泰山區明智路１段２５６巷４弄８號１樓,243.0,0.0,0.0,0


In [44]:
df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')

In [45]:
df_04_1.lon[df_04_1.lon.isna()]=0
df_04_1.lat[df_04_1.lat.isna()]=0
df_04_1.check[df_04_1.check.isna()]=0

In [46]:
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20051121000155,台北市中山區南京都路３段１２９號９樓,104.0,0.0,0.0,0.0
1,20000430000961,新北市八里區商港路96號14樓,249.0,0.0,0.0,0.0
2,20020306000771,彰化縣北斗鎮大新里路１６７巷１９號,521.0,0.0,0.0,0.0
3,20000401009236,新北市林口區中湖路39號之,244.0,0.0,0.0,0.0
4,20031217000642,新北市泰山區明智路１段２５６巷４弄８號１樓,243.0,0.0,0.0,0.0


In [47]:
df_zone=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\郵遞區代碼.xlsx')

In [48]:
df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')

In [49]:
geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
crs = {'init': 'epsg:4326'}
df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
df_04_1.shape

(881, 9)

In [50]:
df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
df_04_1.shape

(881, 13)

In [51]:
df_04_1.head()

,id,addr,zone,lon,lat,check,縣市_left,行政區_left,geometry,index_right,縣市_right,行政區_right,村里
0,20051121000155,台北市中山區南京都路３段１２９號９樓,104.0,0.0,0.0,0.0,台北市,中山區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
1,20000430000961,新北市八里區商港路96號14樓,249.0,0.0,0.0,0.0,新北市,八里區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
2,20020306000771,彰化縣北斗鎮大新里路１６７巷１９號,521.0,0.0,0.0,0.0,彰化縣,北斗鎮,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
3,20000401009236,新北市林口區中湖路39號之,244.0,0.0,0.0,0.0,新北市,林口區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
4,20031217000642,新北市泰山區明智路１段２５６巷４弄８號１樓,243.0,0.0,0.0,0.0,新北市,泰山區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN


In [52]:
# 無 map 無 zone
df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
print(df_04_1_1.shape)

# 無 map 有 zone
df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
print(df_04_1_2.shape)

# 有 map 無有 zone
df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
print(df_04_1_3.shape)

# 有 map 有 zone
df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
print(df_04_1_4.shape)

(8, 13)
(662, 13)
(9, 13)
(202, 13)


In [53]:
df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_1.shape)

df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_2.shape)

(167, 13)
(35, 13)


In [54]:
df_04_1_1=df_04_1_1.loc[:,['id']]

df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_2.columns=['id','縣市','行政區']

df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_3.columns=['id','縣市','行政區','村里']

df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_4_1.columns=['id','縣市','行政區','村里']

df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_4_2.columns=['id','縣市','行政區']

In [55]:
df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
df_04_2.shape

(881, 4)

In [56]:
df_04_2.drop_duplicates(subset=['id']).shape

(881, 4)

In [57]:
df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
df_04_3.shape

(1464, 3)

In [58]:
df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')

In [59]:
df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
print(df_04_3_1.shape)

df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
print(df_04_3_2.shape)

(391, 5)
(1073, 5)


In [60]:
df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
df_04_3_2=df_04_3_2.loc[:,['id']]

In [61]:
df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
print(df_04.shape)
print(df_04.drop_duplicates(subset=['id']).shape)

(2345, 4)
(2345, 4)


In [62]:
result=pd.concat([df_01,df_04]).reset_index(drop=True)
result.loc[result['縣市'].isna(),['縣市']]='NA'
result.loc[result['行政區'].isna(),['行政區']]='NA'
result.loc[result['村里'].isna(),['村里']]='NA'
print(result.shape)
result.head()

(14384, 7)


,id,addr,lon,lat,縣市,行政區,村里
0,20010303019078,新北市淡水區新興里7鄰新春街136號 ...,121.441461,25.181664,新北市,淡水區,新興里
1,19961005000002,臺北市士林區天和里4鄰天母東路77之2號 ...,121.535273,25.118405,台北市,士林區,天和里
2,20110403000689,臺北市信義區西村里9鄰基隆路一段378號 ...,121.560424,25.034843,台北市,信義區,西村里
3,20011220001189,高雄市大社區大社里13鄰大社路28巷10號 ...,120.345319,22.732471,高雄市,大社區,大社里
4,20110712001784,桃園市蘆竹區上興里5鄰上興路74巷101號 ...,121.263671,25.019795,桃園市,蘆竹區,上興里


In [63]:
have=result.shape[0]
lost=(result['村里']=='NA').sum()
have_share=1-lost/have
print(site[0],'have_share',have_share)

1G30林口中山店 have_share 0.8492074527252502


In [64]:
df_sp01=pd.merge(df02,result)
df_sp01['行政區2']=df_sp01['行政區']
def replace_place(x):
    religion=['林口區','龜山區']
    if x =='NA':
        x=x
        return x
    elif x not in religion:
        x='其他'
        return x
    else:
        return x

df_sp01['行政區2']=df_sp01['行政區2'].apply(replace_place)
df_sp02=df_sp01.groupby(['type','行政區2'])['id'].count().reset_index()
df_sp02

,type,行政區2,id
0,type1:new,NA,50
1,type1:new,其他,2
2,type1:new,林口區,12
3,type1:new,龜山區,3
4,type2:active,NA,95
5,type2:active,其他,479
6,type2:active,林口區,515
7,type2:active,龜山區,63
8,type3:1yr~3yr,NA,1
9,type3:1yr~3yr,其他,63


In [65]:
pop['縣市']=pop['縣市'].str.replace('\xa0','')
pop['行政區']=pop['行政區'].str.replace('\xa0','')
pop['村里']=pop['村里'].str.replace('\xa0','')

In [66]:
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))
pop['縣市']=list(pop['縣市'].apply(lambda x:x.replace('臺','台')))
pop['行政區']=list(pop['行政區'].apply(lambda x:x.replace('臺','台')))
pop['村里']=list(pop['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('欍','瓊'))


pop['村里']=pop['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))

In [67]:
result[-(result['行政區']+result['村里']).isin(gdf['行政區']+gdf['村里'])]['村里'].unique()

array(['NA'], dtype=object)

In [68]:
result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

In [69]:
result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
result2.columns=['縣市','行政區','村里','會員數']
result2.shape

(2454, 4)

In [70]:
result.shape[0]

14384

In [71]:
result2=pd.merge(pop,result2,how='outer')

In [72]:
result2=pd.merge(pop,result2,how='outer')
result2=result2[-(-result2['村里'].isin(pop['村里'])&-(result2['村里']=='NA'))].reset_index(drop=True)
result2.shape

(7878, 6)

In [73]:
result2[-(result2['行政區']+result2['村里']).isin(gdf['行政區']+gdf['村里'])]['村里'].unique()

array(['NA'], dtype=object)

In [74]:
result2['會員數'].sum()

14384.0

In [75]:
gdf_final=pd.merge(gdf,result2,how='right')

In [76]:
gdf_final['會員數'][gdf_final['會員數'].isna()]=0

In [77]:
gdf_final['site']=gdf_final['行政區']+'\n'+gdf_final['村里']
gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
print(gdf_final.shape)
gdf_final.head()

(7878, 9)


,縣市,行政區,村里,geometry,戶口,人口,會員數,site,ratio
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",1119.0,2871.0,0.0,林園區\n中門里,0.000000
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492...",821.0,2049.0,0.0,林園區\n港埔里,0.000000
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577...",1380.0,3213.0,1.0,旗津區\n上竹里,0.000725
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557...",1065.0,2881.0,0.0,旗津區\n中興里,0.000000
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141...",114.0,443.0,0.0,南竿鄉\n津沙村,0.000000


In [78]:
north1=['新北市','台北市','桃園市']

In [79]:
area=[north1]

In [80]:
print(gdf_final.shape)

location=area[0]
gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
print(gdf_final_sub.shape)

gdf_final_sub_out=gdf_final.loc[-gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
print(gdf_final_sub_out.shape)

(7878, 9)
(2041, 6)
(5837, 6)


In [81]:
NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']=='NA']['會員數'].sum()
NA_value

not_NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']!='NA']['會員數'].sum()
not_NA_value

1162.0

In [82]:
NA_value

1081.0

In [83]:
df_final06=gdf_final_sub.groupby(['縣市','行政區'])['會員數','戶口'].sum().reset_index()
df_final06['ratio']=df_final06['會員數']/df_final06['戶口']

NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']=='NA']['會員數'].sum()

not_NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']!='NA']['會員數'].sum()

df_final06=pd.concat([df_final06,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
print(df_final06.shape)
df_final06.head()

(56, 5)


,縣市,行政區,會員數,戶口,ratio
0,台北市,中山區,262.0,100900.0,0.002597
1,台北市,中正區,115.0,65782.0,0.001748
2,台北市,信義區,138.0,89132.0,0.001548
3,台北市,內湖區,257.0,109817.0,0.002340
4,台北市,北投區,171.0,98171.0,0.001742


In [84]:
df_final06['會員數'].sum()==gdf_final['會員數'].sum()

True

In [85]:
df_final07=gdf_final_sub.copy()

NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']=='NA']['會員數'].sum()

not_NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']!='NA']['會員數'].sum()

df_final07=pd.concat([df_final07,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
print(df_final07.shape)
df_final07.head()

(2043, 6)


,縣市,行政區,村里,會員數,戶口,ratio
42,桃園市,大溪區,仁義里,0.0,1374.0,0.000000
43,桃園市,八德區,瑞德里,1.0,1572.0,0.000636
872,桃園市,龜山區,楓樹里,11.0,1669.0,0.006591
873,桃園市,桃園區,大興里,5.0,2609.0,0.001916
874,桃園市,桃園區,忠義里,6.0,1657.0,0.003621


In [86]:
df_final01['店點']=site[0]
df_final01=df_final01.set_index('店點').reset_index()

df_final02['店點']=site[0]
df_final02=df_final02.set_index('店點').reset_index()

df_final03['店點']=site[0]
df_final03=df_final03.set_index('店點').reset_index()

df_final04['店點']=site[0]
df_final04=df_final04.set_index('店點').reset_index()

df_final05['店點']=site[0]
df_final05=df_final05.set_index('店點').reset_index()

df_final06['店點']=site[0]
df_final06=df_final06.set_index('店點').reset_index()

df_final07['店點']=site[0]
df_final07=df_final07.set_index('店點').reset_index()

In [87]:
df_final05_2=df_final05.copy()
df_final05_2['']=''
df_final05_2=df_final05_2.set_index('').reset_index()


df_final06_2=df_final06.copy()
df_final06_2['']=''
df_final06_2=df_final06_2.set_index('').reset_index()

df_final07_2=df_final07.copy()
df_final07_2['']=''
df_final07_2=df_final07_2.set_index('').reset_index()

In [88]:
df_final05a=df_final05.copy()
df_final06a=df_final06.copy()
df_final07a=df_final07.copy()
df_final05b=df_final05_2.copy()
df_final06b=df_final06_2.copy()
df_final07b=df_final07_2.copy()

In [89]:
df_final06

,店點,縣市,行政區,會員數,戶口,ratio
0,1G30林口中山店,台北市,中山區,262.0,100900.0,0.002597
1,1G30林口中山店,台北市,中正區,115.0,65782.0,0.001748
2,1G30林口中山店,台北市,信義區,138.0,89132.0,0.001548
3,1G30林口中山店,台北市,內湖區,257.0,109817.0,0.002340
4,1G30林口中山店,台北市,北投區,171.0,98171.0,0.001742
5,1G30林口中山店,台北市,南港區,71.0,48036.0,0.001478
6,1G30林口中山店,台北市,士林區,258.0,108098.0,0.002387
7,1G30林口中山店,台北市,大同區,91.0,52045.0,0.001748
8,1G30林口中山店,台北市,大安區,197.0,121420.0,0.001622
9,1G30林口中山店,台北市,文山區,118.0,107598.0,0.001097


In [90]:
site=['1G30林口中山店','1G32桃園大業店','1G33竹北文興店','1G34蘆洲集賢店','1G35板橋新埔店','1G36新營金華店','1G37永康復國店','1G38大安安和店','1G39頭份中央店','1G40三民澄清店']
start=['190726','190903','191123','191204','200328','200321','200411','200530','200711','200912']

north1=['新北市','台北市','桃園市']
north2=['新竹縣','新竹市','桃園市','新北市']
north3=['新竹縣','新竹市','桃園市']
north4=['新竹縣','新竹市','苗栗縣']
west=['南投縣','彰化縣','台中市']
south1=['台南市','嘉義縣','嘉義市']
south2=['高雄市','屏東縣']
east=[]
area=[north1,north2,north3,north1,north1,south1,south1,north1,north4,south2]

In [91]:
print(int(date[0][2:])*100+40)
end_site=(np.array(list(map(int,start)))<int(date[0][2:])*100+40).sum()
end_site

200940


10

In [92]:
for i in range(1,end_site):
    print(site[i])

1G32桃園大業店
1G33竹北文興店
1G34蘆洲集賢店
1G35板橋新埔店
1G36新營金華店
1G37永康復國店
1G38大安安和店
1G39頭份中央店
1G40三民澄清店


# 看不懂的迴圈起始點

In [98]:
for a in range(1,end_site):
    os.chdir('C:/Users/RT009223/TestriteGroup/滲透率專案/01_raw data/TLW社區店/'+site[a]+'/'+date[0])
    print(site[a],date[0])
    df01=pd.read_csv(r'ff交易紀錄.csv',header=None,names=['id','site','date','AMT','trans_cnt','tick_cnt','card_type'])
    print(df01.shape)
    df01.head()
    df01['temp']=0
    df01.loc[df01.card_type=='05',['temp']]=1
    df01_2=df01.groupby(['site','temp'])['AMT'].sum().reset_index(name='AMT')
    df01_2['share']=df01_2.AMT/df01_2.AMT.sum()
    df_final01_another=df01_2.copy()


    df02=pd.read_csv(r'一般卡會員活躍程度.csv',header=None,names=['id','type'])
    print('當月一般卡會員',df02.shape)
    df02.head()
    df02.type=df02.type.apply(lambda x:str(x))
    df02.type=df02.type.replace('1','type1:new')
    df02.type=df02.type.replace('2','type2:active')
    df02.type=df02.type.replace('3','type3:1yr~3yr')
    df02.type=df02.type.replace('4','type4:3yr~')
    df02_2=df02.groupby(['type'])['id'].count().reset_index(name='n')
    df02_2['share']=df02_2.n/df02_2.n.sum()
    df_final02_another=df02_2.copy()

    df03=pd.read_csv(r'一般卡會員會員樣貌.csv',header=None,names=['id','name','gender','birth','status','age','age_type'])
    print(df03.shape)
    df03.head()
    df03=pd.merge(df03,df02,on='id')
    df03_2=df03.groupby(['age_type'])['id'].count().reset_index(name='n_all')
    df03_2['share_all']=df03_2.n_all/df03_2.n_all.sum()
    df03_3=df03[df03.type=='type1:new'].groupby(['age_type'])['id'].count().reset_index(name='n_new')
    df03_3['share_new']=df03_3.n_new/df03_3.n_new.sum()
    df03_4=pd.merge(df03_2,df03_3,on='age_type',how='outer')
    df03_4=df03_4.fillna(0)
    print(df03_4.share_all.sum())
    print(df03_4.share_new.sum())
    df_final03_another=df03_4.copy()

    df03_5=df03.groupby(['gender'])['id'].count().reset_index(name='n_all')
    df03_5['share_all']=df03_5.n_all/df03_5.n_all.sum()
    df03_6=df03[df03.type=='type1:new'].groupby(['gender'])['id'].count().reset_index(name='n_new')
    df03_6['share_new']=df03_6.n_new/df03_6.n_new.sum()
    df03_7=pd.merge(df03_5,df03_6,on='gender',how='outer')
    df03_7=df03_7.fillna(0)
    print(df03_7.share_all.sum())
    print(df03_7.share_new.sum())
    df_final04_another=df03_7.copy()

    df04=pd.read_csv(r'sales一般卡交易紀錄.csv',header=None,names=['id','card','site','date','sku','class1','class1_name','AMT'])
    print(df04.shape)
    df04.head()
    df04_2=pd.merge(df04,df03,on='id',how='left')
    print(df04_2.shape)
    df04_2.head()
    df04_3=df04_2.groupby(['class1_name'])['id'].nunique().reset_index(name='id_all')
    df04_3['id_all_rate']=df04_3.id_all/df03.shape[0]
    print(df04_3.shape)
    df04_3.head()
    df04_4=df04_2[df04_2.type=='type1:new'].groupby(['class1_name'])['id'].nunique().reset_index(name='id_new')
    df04_4['id_new_rate']=df04_4.id_new/df03[df03.type=='type1:new'].shape[0]
    print(df04_4.shape)
    df04_4.head()
    dept_name=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\TLW社區店\大分類中文名稱.xlsx')
    df04_5=pd.merge(df04_3,df04_4,on='class1_name',how='outer')
    df04_6=pd.merge(dept_name,df04_5,how='left')
    df04_6=df04_6.fillna(0)
    df_final05_another=df04_6.copy()



    df_01=pd.read_csv('一般卡會員(有完整地址)_'+start[a]+'.csv',error_bad_lines=False,header=None)
    df_01.columns=['id','addr','lon','lat']
    print('一般卡會員(有完整地址)',df_01.shape)
    pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
    location=df_01.addr.apply(lambda x:pattern.findall(x))

    addr01=[]
    addr02=[]
    addr03=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0][0])
            addr02.append(i[0][1])
            addr03.append(i[0][2])        
        else:
            addr01.append(np.nan)
            addr02.append(np.nan)
            addr03.append(np.nan)  
    df_01['縣市']=addr01
    df_01['行政區']=addr02
    df_01['村里']=addr03
    df_02=df_01.dropna()
    print(df_02.shape)

    print(gdf.shape)

    gdf.columns=['縣市','行政區','村里','geometry']
    gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
    gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
    gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

    df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
    df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
    df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
    gdf.head()
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

    df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
    df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))



    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

    #df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
    df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

    df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


    df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
    df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
    df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?田里','塩田里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('山?里','山脚里'))

    df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水?村','水祭村'))
    df_02.loc[df_02['行政區']=='北埔鄉',['村里']]=df_02.loc[df_02['行政區']=='北埔鄉',['村里']].apply(lambda x:x.replace('水？村','水祭村'))
    df_02.loc[df_02['行政區']=='瑞芳區',['村里']]=df_02.loc[df_02['行政區']=='瑞芳區',['村里']].apply(lambda x:x.replace('?洞里','濂洞里'))

    df_02.loc[df_02['行政區']=='埔心鄉',['村里']]=df_02.loc[df_02['行政區']=='埔心鄉',['村里']].apply(lambda x:x.replace('埤?村','埤脚村'))
    
    
    #合
    df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

    #不合
    df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_02_1.shape)
    print(df_02_2.shape)
    df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
    df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))
    if df_02_2.shape[0]>0:        
        geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
        crs = {'init': 'epsg:4326'}
        df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
        print(df_02_2.shape)
        df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
        print(df_02_3.shape)
        df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
        df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
        df_02_3.head()
    else:
        df_02_3=df_02_2.copy()
    df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
    print(df_02.shape)
    df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
    print(df_03.shape)
    pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
    location=df_03.addr.apply(lambda x:pattern.findall(x))

    addr01=[]
    addr02=[]
    addr03=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0][0])
            addr02.append(i[0][1])
            addr03.append(i[0][2])        
        else:
            addr01.append(np.nan)
            addr02.append(np.nan)
            addr03.append(np.nan)  
    df_03['縣市']=addr01
    df_03['行政區']=addr02
    df_03['村里']=addr03
    #合
    df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_03_1.shape)

    #不合
    df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_03_2.shape)
    def to_float(x):
        try:
            return float(x)
        except:
            return 0
    df_03_2.lon=df_03_2.lon.apply(to_float)
    df_03_2.lat=df_03_2.lat.apply(to_float)
    if df_03_2.shape[0]>0:
        geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
        crs = {'init': 'epsg:4326'}
        df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
        print(df_03_2.shape)
        df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
        print(df_03_3.shape)
        df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
        df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
        print(df_03_3.shape)
    else:
        df_03_3=df_03_2.copy()
    df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)
    df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
    print('一般卡會員(有完整地址)_new',df_01.shape)
    
    
    df_04=pd.read_excel('一般卡會員(無完整地址)_'+start[a]+'.xlsx')
    print('一般卡會員(無完整地址)',df_04.shape)
    df_04['addr']=df_04.addr.apply(lambda x:str(x))
    df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
    print(df_04_1.shape)
    pattern = re.compile(r'..[市,縣].+[路,街].+')
    location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

    addr01=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0])      
        else:
            addr01.append(np.nan)
    df_04_1.addr=addr01
    df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
    print(df_04_1.shape)
    df_04_1['lon']=float(0)
    df_04_1['lat']=float(0)
    df_04_1['check']=0
    df_04_1.head()


    print(df_get_back.shape)
    df_get_back.head()
    df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')
    df_04_1.lon[df_04_1.lon.isna()]=0
    df_04_1.lat[df_04_1.lat.isna()]=0
    df_04_1.check[df_04_1.check.isna()]=0
    df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')
    geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
    df_04_1.shape
    df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
    print(df_04_1.shape)

    # 無 map 無 zone
    df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
    print(df_04_1_1.shape)

    # 無 map 有 zone
    df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
    print(df_04_1_2.shape)

    # 有 map 無有 zone
    df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
    print(df_04_1_3.shape)

    # 有 map 有 zone
    df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
    print(df_04_1_4.shape)

    df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
    print(df_04_1_4_1.shape)

    df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
    print(df_04_1_4_2.shape)

    df_04_1_1=df_04_1_1.loc[:,['id']]

    df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
    df_04_1_2.columns=['id','縣市','行政區']

    df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
    df_04_1_3.columns=['id','縣市','行政區','村里']

    df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
    df_04_1_4_1.columns=['id','縣市','行政區','村里']

    df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
    df_04_1_4_2.columns=['id','縣市','行政區']

    df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
    print(df_04_2.shape)
    print(df_04_2.drop_duplicates(subset=['id']).shape)
    df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
    print(df_04_3.shape)

    df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')
    df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
    print(df_04_3_1.shape)

    df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
    print(df_04_3_2.shape)
    df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
    df_04_3_2=df_04_3_2.loc[:,['id']]
    df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
    print('一般卡會員(無完整地址)_new',df_04.shape)
    print(df_04.drop_duplicates(subset=['id']).shape)
    result=pd.concat([df_01,df_04]).reset_index(drop=True)
    result.loc[result['縣市'].isna(),['縣市']]='NA'
    result.loc[result['行政區'].isna(),['行政區']]='NA'
    result.loc[result['村里'].isna(),['村里']]='NA'
    print('全部一般卡會員_new',result.shape)
    result.head()
    have=result.shape[0]
    lost=(result['村里']=='NA').sum()
    have_share=1-lost/have
    print(site[a],'have_share',have_share)
    
    ########################
    if site[a]=='1G40三民澄清店':
        df_sp01=pd.merge(df02,result)
        df_sp01['行政區2']=df_sp01['行政區']
        def replace_place(x):
            religion=['三民區','鳳山區','苓雅區','鳥松區']
            if x =='NA':
                x=x
                return x
            elif x not in religion:
                x='其他'
                return x
            else:
                return x

        df_sp01['行政區2']=df_sp01['行政區2'].apply(replace_place)
        df_sp02=df_sp01.groupby(['type','行政區2'])['id'].count().reset_index()
        df_sp02.to_excel('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/三民澄清店來客區域.xlsx',index=False)
    ########################
    
    result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
    result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
    result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
    result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

    result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
    result2.columns=['縣市','行政區','村里','會員數']
    print(result2.shape)
    result2=pd.merge(pop,result2,how='outer')
    result2=result2[-(-result2['村里'].isin(pop['村里'])&-(result2['村里']=='NA'))].reset_index(drop=True)
    print(result2.shape)

    gdf_final=pd.merge(gdf,result2,how='right')
    gdf_final['會員數'][gdf_final['會員數'].isna()]=0
    gdf_final['site']=gdf_final['行政區']+'\n'+gdf_final['村里']
    gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
    print(gdf_final.shape)
    gdf_final.head()
    location=area[a]
    gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
    print(gdf_final_sub.shape)

    df_final06_another=gdf_final_sub.groupby(['縣市','行政區'])['會員數','戶口'].sum().reset_index()
    df_final06_another['ratio']=df_final06_another['會員數']/df_final06_another['戶口']
    NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']=='NA']['會員數'].sum()
    not_NA_value=gdf_final_sub_out[gdf_final_sub_out['行政區']!='NA']['會員數'].sum()
    df_final06_another=pd.concat([df_final06_another,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
    print(df_final06_another.shape)

    df_final07_another=gdf_final_sub.copy()
    NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']=='NA']['會員數'].sum()
    not_NA_value=gdf_final_sub_out[gdf_final_sub_out['村里']!='NA']['會員數'].sum()
    df_final07_another=pd.concat([df_final07_another,pd.DataFrame({'縣市':['其他地區','其他地區NA'],'會員數':[not_NA_value,NA_value]})])
    print(df_final07_another.shape)

    df_final01_another['店點']=site[a]
    df_final01_another=df_final01_another.set_index('店點').reset_index()

    df_final02_another['店點']=site[a]
    df_final02_another=df_final02_another.set_index('店點').reset_index()

    df_final03_another['店點']=site[a]
    df_final03_another=df_final03_another.set_index('店點').reset_index()

    df_final04_another['店點']=site[a]
    df_final04_another=df_final04_another.set_index('店點').reset_index()

    df_final05_another['店點']=site[a]
    df_final05_another=df_final05_another.set_index('店點').reset_index()

    df_final06_another['店點']=site[a]
    df_final06_another=df_final06_another.set_index('店點').reset_index()

    df_final07_another['店點']=site[a]
    df_final07_another=df_final07_another.set_index('店點').reset_index()
    
    df_final05_2=df_final05_another.copy()
    df_final05_2['']=''
    df_final05_2=df_final05_2.set_index('').reset_index()
    
    df_final06_2=df_final06_another.copy()
    df_final06_2['']=''
    df_final06_2=df_final06_2.set_index('').reset_index()

    df_final07_2=df_final07_another.copy()
    df_final07_2['']=''
    df_final07_2=df_final07_2.set_index('').reset_index()

    df_final01=pd.concat([df_final01,df_final01_another])
    df_final02=pd.concat([df_final02,df_final02_another])
    df_final03=pd.concat([df_final03,df_final03_another])
    df_final04=pd.concat([df_final04,df_final04_another])
    
    df_final05a=pd.concat([df_final05a,df_final05_another])
    df_final06a=pd.concat([df_final06a,df_final06_another])
    df_final07a=pd.concat([df_final07a,df_final07_another])

    df_final05b=pd.concat([df_final05b, df_final05_2], axis=1)
    df_final06b=pd.concat([df_final06b, df_final06_2], axis=1)
    df_final07b=pd.concat([df_final07b, df_final07_2], axis=1)
    print('------------------------------------------------------')
    print('------------------------------------------------------')
    print('------------------------------------------------------')

1G32桃園大業店 202009
(2115, 7)
當月一般卡會員 (1609, 2)
(1609, 7)
1.0
1.0
1.0
1.0
(4911, 8)
(4911, 15)
(25, 3)
(21, 3)
一般卡會員(有完整地址) (8589, 4)
(8588, 7)
(7760, 4)
(8588, 7)
(0, 7)
(8588, 7)
(1, 7)
(1, 7)
(0, 7)
一般卡會員(有完整地址)_new (8589, 7)
一般卡會員(無完整地址) (2650, 3)
(1061, 3)
(926, 3)
(133014, 4)
(926, 13)
(5, 13)
(794, 13)
(4, 13)
(123, 13)
(97, 13)
(26, 13)
(926, 4)
(926, 4)
(1724, 3)
(244, 5)
(1480, 5)
一般卡會員(無完整地址)_new (2650, 4)
(2650, 4)
全部一般卡會員_new (11239, 7)
1G32桃園大業店 have_share 0.7732004626746152
(1747, 4)
(7849, 6)
(7849, 9)
(1883, 6)
(60, 5)
(1885, 6)
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
1G33竹北文興店 202009
(2311, 7)
當月一般卡會員 (1846, 2)
(1846, 7)
0.9999999999999999
1.0
0.9999999999999999
1.0
(6571, 8)
(6571, 15)
(27, 3)
(22, 3)
一般卡會員(有完整地址) (12467, 4)
(12464, 7)
(7760, 4)
(12462, 7)
(2, 7)
(2, 8)
(2, 12)
(12464, 7)
(3, 7)
(2, 7)
(1, 7)
(1, 8)
(1, 12)
(1, 7)
一般卡會員(有完整地址)_new

In [94]:
date[0]

'202009'

In [95]:
date[0][2:]

'2009'

In [96]:
writer = pd.ExcelWriter('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'+date[0][2:]+'/社區店分析_分析用.xlsx', engine='xlsxwriter')

df_final01.to_excel(writer, sheet_name='臨時卡業績占比',index=0)
df_final02.to_excel(writer, sheet_name='會員組成',index=0)
df_final03.to_excel(writer, sheet_name='年齡',index=0)
df_final04.to_excel(writer, sheet_name='性別',index=0)
df_final05a.to_excel(writer, sheet_name='品類',index=0)
df_final06a.to_excel(writer, sheet_name='區滲透',index=0)
df_final07a.to_excel(writer, sheet_name='里滲透',index=0)

writer.save()

In [97]:
writer = pd.ExcelWriter('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'+date[0][2:]+'/社區店分析('+date[0]+')_Iris用.xlsx', engine='xlsxwriter')

df_final01.to_excel(writer, sheet_name='臨時卡業績占比',index=0)
df_final02.to_excel(writer, sheet_name='會員組成',index=0)
df_final03.to_excel(writer, sheet_name='年齡',index=0)
df_final04.to_excel(writer, sheet_name='性別',index=0)
df_final05b.to_excel(writer, sheet_name='品類',index=0)
df_final06b.to_excel(writer, sheet_name='區滲透',index=0)
df_final07b.to_excel(writer, sheet_name='里滲透',index=0)

writer.save()

In [93]:
location=north1
gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
gdf_final_sub.shape

(2007, 6)